### Signal Processing Showcase and Exploration 
Extracting the envelope through different data processing methodologies

The selected movements for our database construction are [0, 5, 6, 9, 10, 13, 17] of E3 of DB1.

- Cilindric: 5
- CloseHand: 10
- Handle: 17
- Pinch: 6
- PointTripod: 9
- Tripod: 13

### Importing the data

In [ ]:
# Plot millis vs s1 for "Cilindric" grasp across all subjects
print("Plotting millis vs s1 for 'Cilindric' grasp across all subjects...")

# First, check which subjects have "Cilindric" grasp and s1 column
subjects_with_cilindric = []
for subject_name, df in subject_dataframes.items():
    subject_only = subject_name.split('_')[0]
    has_cilindric = "Cilindric" in df['Grasp'].unique()
    has_s1 = 's1' in df.columns
    
    if has_cilindric and has_s1:
        subjects_with_cilindric.append((subject_only, subject_name))
        print(f"✓ {subject_only}: Has Cilindric grasp and s1 column")
    else:
        print(f"✗ {subject_only}: {'Missing Cilindric grasp' if not has_cilindric else ''}"
              f"{' and ' if not has_cilindric and not has_s1 else ''}"
              f"{'Missing s1 column' if not has_s1 else ''}")

if not subjects_with_cilindric:
    print("No subjects have both 'Cilindric' grasp and 's1' column.")
else:
    # Create plot with subplots for each subject
    n_subjects = len(subjects_with_cilindric)
    fig, axes = plt.subplots(n_subjects, 1, figsize=(14, 4 * n_subjects), sharex=True)
    
    # Handle case where there's only one subject (axes won't be an array)
    if n_subjects == 1:
        axes = [axes]
    
    # Plot each subject's data
    for i, (subject_only, subject_name) in enumerate(subjects_with_cilindric):
        # Get the subject's dataframe and filter for Cilindric grasp
        df = subject_dataframes[subject_name]
        cilindric_df = df[df['Grasp'] == 'Cilindric']
        
        # Plot millis vs s1
        axes[i].plot(cilindric_df['millis'], cilindric_df['s1'], 
                    label=f"{subject_only} - Cilindric", 
                    color=f'C{i}', linewidth=1.5)
        
        # Add horizontal line at zero
        axes[i].axhline(y=0, color='gray', linestyle='-', alpha=0.3)
        
        # Customize subplot
        axes[i].set_title(f"{subject_only} - Cilindric Grasp")
        axes[i].set_ylabel('s1 Value')
        axes[i].grid(True, alpha=0.3)
        axes[i].legend()
    
    # Set common x-axis label
    axes[-1].set_xlabel('Time (millis)')
    
    # Add an overall title
    plt.suptitle('s1 Values for Cilindric Grasp Across Subjects', fontsize=16)
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)  # Make room for suptitle
    plt.show()
    
    # Also create an interactive plotly visualization combining all subjects in one graph
    plot_data = []
    for subject_only, subject_name in subjects_with_cilindric:
        df = subject_dataframes[subject_name]
        cilindric_df = df[df['Grasp'] == 'Cilindric']
        
        # Create a copy to avoid modifying original
        temp_df = pd.DataFrame({
            'Subject': subject_only,
            'Time (ms)': cilindric_df['millis'],
            's1': cilindric_df['s1']
        })
        
        plot_data.append(temp_df)
    
    if plot_data:
        combined_df = pd.concat(plot_data, ignore_index=True)
        
        fig = px.line(combined_df, x='Time (ms)', y='s1', 
                     color='Subject', 
                     title='s1 Values for Cilindric Grasp Across Subjects',
                     labels={'s1': 's1 Value', 'Time (ms)': 'Time (ms)'},
                     height=600)
        
        # Customize layout
        fig.update_layout(
            xaxis_title="Time (ms)",
            yaxis_title="s1 Value",
            template="plotly_white",
            legend_title="Subject"
        )
        
        # Add horizontal line at zero
        fig.add_shape(
            type="line", line=dict(dash="solid", color="gray", width=1),
            y0=0, y1=0, x0=0, x1=1, xref="paper", yref="y"
        )
        
        # Display the interactive plot
        fig.show()

In [ ]:
# Enhanced normalization with outlier detection and correction
print("Detecting and correcting outliers before normalization...")

# Track changes for reporting
outlier_changes = []
rolling_window = 20  # Size of rolling window for calculating local average
std_threshold = 10    # Number of standard deviations to consider as outlier

# Find all sensor columns across all dataframes
sensor_columns = []
for subject_name, df in subject_dataframes.items():
    # Get column names that start with 's' followed by a number
    sensor_cols = [col for col in df.columns if re.match(r's[1-8]$', col)]
    sensor_columns.extend(sensor_cols)

# Get unique sensor column names
sensor_columns = sorted(set(sensor_columns))
print(f"Found sensor columns: {', '.join(sensor_columns)}")

# Process each subject's dataframe
for subject_name, df in subject_dataframes.items():
    subject_only = subject_name.split('_')[0]
    print(f"\nProcessing outlier detection for {subject_only}...")
    
    # Get unique grasp types
    grasp_types = df['Grasp'].unique()
    
    # Process each grasp type separately to avoid contamination between grasps
    for grasp in grasp_types:
        grasp_mask = df['Grasp'] == grasp
        grasp_indices = df[grasp_mask].index
        
        if len(grasp_indices) == 0:
            continue
            
        print(f"  Checking {grasp} grasp...")
        
        # Process each sensor column if it exists
        for sensor in sensor_columns:
            if sensor in df.columns:
                # Only process the current grasp's data
                sensor_data = df.loc[grasp_mask, sensor].copy()
                
                if len(sensor_data) <= rolling_window:
                    # Not enough data for this grasp to calculate rolling stats
                    continue
                
                # Calculate rolling mean and std
                rolling_mean = sensor_data.rolling(window=rolling_window, min_periods=1).mean()
                rolling_std = sensor_data.rolling(window=rolling_window, min_periods=1).std()
                
                # Initialize a mask for potential outliers
                is_outlier = np.zeros(len(sensor_data), dtype=bool)
                
                # For each point, check if it's too far from the rolling mean
                for i in range(rolling_window, len(sensor_data)):
                    current_value = sensor_data.iloc[i]
                    mean_value = rolling_mean.iloc[i-1]  # Use previous window mean
                    std_value = rolling_std.iloc[i-1]    # Use previous window std
                    
                    # Skip if std is too small (to avoid division by near-zero)
                    if std_value < 0.001:
                        continue
                    
                    # Check if point is beyond threshold
                    z_score = abs(current_value - mean_value) / std_value
                    if z_score > std_threshold:
                        is_outlier[i] = True
                        
                        # Record the change
                        actual_idx = sensor_data.index[i]
                        outlier_changes.append({
                            'Subject': subject_only,
                            'Grasp': grasp,
                            'Sensor': sensor,
                            'Index': actual_idx,
                            'Original': current_value,
                            'Mean': mean_value,
                            'StdDev': std_value,
                            'Z-score': z_score
                        })
                        
                        # Replace the outlier with the rolling mean
                        df.loc[actual_idx, sensor] = mean_value
    
    # Report the number of outliers found for this subject
    subject_outliers = [c for c in outlier_changes if c['Subject'] == subject_only]
    if subject_outliers:
        print(f"  ⚠️ Found and corrected {len(subject_outliers)} outliers")
    else:
        print(f"  ✅ No outliers detected")

# Create a summary dataframe with outlier information
if outlier_changes:
    outlier_df = pd.DataFrame(outlier_changes)
    
    # Display summary statistics
    print("\n===== Outlier Correction Summary =====")
    
    # Count outliers by subject and grasp
    by_subject_grasp = outlier_df.groupby(['Subject', 'Grasp']).size().unstack(fill_value=0)
    print("\nOutliers by Subject and Grasp:")
    display(by_subject_grasp)
    
    # Count outliers by sensor
    by_sensor = outlier_df.groupby('Sensor').size()
    print("\nOutliers by Sensor:")
    display(by_sensor)
    
    # Show a sample of the corrected outliers
    print("\nSample of corrected outliers:")
    display(outlier_df.head(10))
else:
    print("\n✅ No outliers detected across all subjects")

# Now proceed with normalization using the corrected data
print("\n===== Starting Normalization Process =====")
print("Normalizing sensor data (s1-s8) columns using 95th percentile...")

# Normalize each sensor column in each subject's dataframe
for subject_name, df in subject_dataframes.items():
    subject_only = subject_name.split('_')[0]
    print(f"Normalizing sensor data for {subject_only}...")
    
    # Process each sensor column if it exists
    for sensor in sensor_columns:
        if sensor in df.columns:
            # Use 95th percentile of absolute values for normalization
            p95_value = np.percentile(df[sensor].abs(), 95)
            
            if p95_value > 0:  # Avoid division by zero
                # Create a new normalized column
                normalized_col = f"{sensor}_norm"
                df[normalized_col] = df[sensor] / p95_value
                print(f"  Normalized {sensor}: 95th percentile = {p95_value:.2f}")
            else:
                print(f"  Skipping {sensor}: 95th percentile is 0")

# Optional: Plot a few examples of before/after outlier correction
if outlier_changes:
    # Find a subject/grasp/sensor with the most outliers to visualize
    outlier_counts = outlier_df.groupby(['Subject', 'Grasp', 'Sensor']).size().sort_values(ascending=False)
    
    if not outlier_counts.empty:
        # Get the combination with the most outliers
        top_outlier = outlier_counts.index[0]
        subject, grasp, sensor = top_outlier
        
        print(f"\n===== Visualizing Outlier Correction Example =====")
        print(f"Showing {sensor} data for {subject}, {grasp} grasp (had {outlier_counts[0]} outliers)")
        
        # Find the matching subject folder
        matching_folders = [name for name in subject_dataframes.keys() if name.split('_')[0] == subject]
        if matching_folders:
            subject_name = matching_folders[0]
            df = subject_dataframes[subject_name]
            
            # Get indices of outliers for this combination
            relevant_outliers = outlier_df[
                (outlier_df['Subject'] == subject) & 
                (outlier_df['Grasp'] == grasp) & 
                (outlier_df['Sensor'] == sensor)
            ]['Index'].values
            
            # Get the data for this grasp
            grasp_data = df[df['Grasp'] == grasp].copy()
            
            if len(grasp_data) > 0:
                # Create a visualization
                plt.figure(figsize=(15, 8))
                
                # Plot the sensor data highlighting the corrected points
                plt.plot(grasp_data['millis'], grasp_data[sensor], 
                         label=f'{sensor} (Corrected)', color='blue', linewidth=1)
                
                # Mark the locations of the outliers
                if len(relevant_outliers) > 0:
                    outlier_times = grasp_data.loc[grasp_data.index.isin(relevant_outliers), 'millis']
                    outlier_values = [float(o['Mean']) for o in outlier_changes if o['Index'] in relevant_outliers]
                    original_values = [float(o['Original']) for o in outlier_changes if o['Index'] in relevant_outliers]
                    
                    # Plot the original values of outliers
                    plt.scatter(outlier_times, original_values, color='red', s=50, label='Original Outliers')
                    
                    # Connect original to corrected with lines
                    for i in range(len(outlier_times)):
                        plt.plot([outlier_times.iloc[i], outlier_times.iloc[i]], 
                                [original_values[i], outlier_values[i]], 
                                color='red', linestyle='--', alpha=0.5)
                
                plt.title(f'Outlier Correction for {sensor} - {subject}, {grasp} Grasp')
                plt.xlabel('Time (ms)')
                plt.ylabel('Sensor Value')
                plt.grid(True, alpha=0.3)
                plt.legend()
                plt.tight_layout()
                plt.show()

### Automatic baseline labeling

In [ ]:
# Enhanced grasp detection with clear visualization of start/end points
print("Implementing enhanced grasp detection with start/end visualization...")

# Define mapping from grasp names to numeric values E3 from DB1
grasp_mapping = {
    'Cilindric': 5,
    'CloseHand': 10, 
    'Handle': 17,
    'Pinch': 6,
    'PointTripod': 9,
    'Tripod': 13
}

# Define parameters for detection
detection_params = {
    'moving_avg_window': 50,     # Window size for smoothing (in samples)
    'std_threshold_factor': 2,  # Number of standard deviations above baseline for activation
    'min_activation_time': 500,   # Minimum activation duration (ms)
    'cooldown_time': 200,         # Time below threshold before deactivation (ms)
    'onset_marker_size': 100,     # Size of onset marker in plot
    'offset_marker_size': 100     # Size of offset marker in plot
}

# Function to detect grasps using dynamic thresholding
def detect_grasps_dynamic(df, grasp_type, grasp_value, sensor_cols, params=detection_params):
    """Enhanced grasp detection using adaptive thresholding and signal features"""
    grasp_df = df[df['Grasp'] == grasp_type].copy()
    if len(grasp_df) == 0:
        return None
    
    # Initialize output columns
    grasp_df['grasp_active'] = 0       # Binary activation status
    grasp_df['onset_marker'] = 0       # Will be 1 at onset points
    grasp_df['offset_marker'] = 0      # Will be 1 at offset points
    grasp_df['detection_threshold'] = 0 # Store the threshold used
    grasp_df['grasp_label'] = 0        # Store the numeric grasp label
    
    # Compute activation score across sensors (using normalized sensors)
    norm_sensors = [col for col in grasp_df.columns if col.endswith('_norm') 
                    and col.startswith('s') and len(col) <= 8]
    
    if not norm_sensors:
        print(f"  No normalized sensor columns found for {grasp_type}")
        return None
        
    # Create an activation score (mean of absolute normalized values)
    grasp_df['activation_score'] = grasp_df[norm_sensors].abs().mean(axis=1)
    
    # Apply moving average to smooth the activation score
    window_size = params['moving_avg_window']
    grasp_df['smooth_activation'] = grasp_df['activation_score'].rolling(
        window=window_size, center=True, min_periods=1).mean()
    
    # Compute adaptive threshold based on baseline + standard deviation
    # Use first 10% of data as baseline if available
    baseline_size = max(int(len(grasp_df) * 0.12), 1)
    baseline = grasp_df['smooth_activation'].iloc[:baseline_size]
    baseline_mean = baseline.mean()
    baseline_std = baseline.std()
    
    # Threshold = mean + factor * std
    threshold = baseline_mean + params['std_threshold_factor'] * baseline_std
    # Store the threshold for visualization
    grasp_df['detection_threshold'] = threshold
    
    # State tracking
    active = False
    time_activated = 0
    time_below_threshold = 0
    onset_points = []
    offset_points = []
    
    # Process each row for state changes
    indices = grasp_df.index
    for i in range(len(grasp_df)):
        row = grasp_df.iloc[i]
        current_time = row['millis']
        is_above_threshold = row['smooth_activation'] > threshold
        
        # State machine logic
        if active:
            if not is_above_threshold:
                # Increment time below threshold
                time_below_threshold += 50  # Assume 50ms between samples
            else:
                # Reset cooldown if signal goes above threshold again
                time_below_threshold = 0
                
            # Check if we should deactivate
            duration_active = current_time - time_activated
            if (duration_active >= params['min_activation_time'] and 
                time_below_threshold >= params['cooldown_time']):
                active = False
                grasp_df.loc[indices[i], 'grasp_active'] = 0
                grasp_df.loc[indices[i], 'grasp_label'] = 0
                grasp_df.loc[indices[i], 'offset_marker'] = 1
                offset_points.append(i)
            else:
                # Keep active
                grasp_df.loc[indices[i], 'grasp_active'] = 1
                grasp_df.loc[indices[i], 'grasp_label'] = grasp_value
        else:
            # Not active, check if we should activate
            if is_above_threshold:
                active = True
                time_activated = current_time
                time_below_threshold = 0
                grasp_df.loc[indices[i], 'grasp_active'] = 1
                grasp_df.loc[indices[i], 'grasp_label'] = grasp_value
                grasp_df.loc[indices[i], 'onset_marker'] = 1
                onset_points.append(i)
    
    print(f"  Detected {len(onset_points)} grasp onset events for {grasp_type} (label: {grasp_value})")
    return grasp_df

# Create output directory for enhanced plots
output_dir = os.path.join(current_dir, 'enhanced_grasp_plots')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created output directory: {output_dir}")

# Process subjects and visualize detection
for subject_name, df in subject_dataframes.items():
    subject_only = subject_name.split('_')[0]
    print(f"\nProcessing enhanced grasp detection for {subject_only}")
    
    # Get normalized sensor columns
    norm_sensors = [col for col in df.columns if col.endswith('_norm') and col.startswith('s')]
    sensor_cols = sorted(set(col.split('_')[0] for col in norm_sensors))
    
    if not norm_sensors:
        print(f"  Error: No normalized sensor columns found for {subject_only}. Skipping.")
        continue
    
    # Get unique grasp types for this subject
    unique_grasps = df['Grasp'].unique()
    print(f"  Found {len(unique_grasps)} grasp types: {', '.join(unique_grasps)}")
    
    # Process each grasp type
    processed_dfs = []
    for grasp_type in unique_grasps:
        print(f"  Processing grasp: {grasp_type}")
        grasp_value = grasp_mapping.get(grasp_type, 0)
        
        # Apply enhanced detection
        grasp_df = detect_grasps_dynamic(df, grasp_type, grasp_value, sensor_cols)
        
        if grasp_df is not None:
            processed_dfs.append(grasp_df)
            
            # Create and save enhanced plot for this grasp
            plt.figure(figsize=(16, 12))
            
            # 1. Plot normalized sensor values
            plt.subplot(4, 1, 1)
            for i, sensor in enumerate(norm_sensors[:8]):  # Limit to 8 sensors for readability
                plt.plot(grasp_df['millis'], grasp_df[sensor], 
                         label=f'{sensor.split("_")[0]}', alpha=0.7, linewidth=0.8)
            plt.title(f"Normalized Sensor Values - {subject_only}, {grasp_type} (Label: {grasp_value})")
            plt.ylabel("Normalized Value")
            plt.grid(True, alpha=0.3)
            plt.legend(loc='upper right', ncol=min(8, len(norm_sensors)))
            
            # 2. Plot activation score with threshold
            plt.subplot(4, 1, 2)
            plt.plot(grasp_df['millis'], grasp_df['smooth_activation'], 
                     label='Activation Score (Smoothed)', color='blue', linewidth=1.5)
            plt.plot(grasp_df['millis'], grasp_df['detection_threshold'], 
                     label='Detection Threshold', color='red', linestyle='--')
            plt.fill_between(grasp_df['millis'], 
                             grasp_df['detection_threshold'], 
                             grasp_df['smooth_activation'],
                             where=(grasp_df['smooth_activation'] >= grasp_df['detection_threshold']),
                             color='green', alpha=0.3, label='Above Threshold')
            plt.title(f"Activation Score and Threshold - {subject_only}, {grasp_type}")
            plt.ylabel("Score")
            plt.grid(True, alpha=0.3)
            plt.legend(loc='upper right')
            
            # 3. Plot detected grasp state with numeric labels
            plt.subplot(4, 1, 3)
            plt.step(grasp_df['millis'], grasp_df['grasp_active'], where='post',
                     label='Grasp Active', color='green', linewidth=2)
            
            # Add a second plot line to show the actual label values when active
            active_mask = grasp_df['grasp_active'] > 0
            if active_mask.any():
                plt.plot(grasp_df.loc[active_mask, 'millis'], 
                         grasp_df.loc[active_mask, 'grasp_label'] / grasp_value, 
                         'g--', alpha=0.5, linewidth=1.5, 
                         label=f'Label: {grasp_value}')
            
            # Mark onset points with green triangles
            onset_indices = grasp_df[grasp_df['onset_marker'] == 1].index
            onset_times = grasp_df.loc[onset_indices, 'millis']
            plt.scatter(onset_times, [1] * len(onset_times), 
                        marker='^', color='lime', s=detection_params['onset_marker_size'],
                        label='Grasp Onset', zorder=5)
            
            # Mark offset points with red triangles
            offset_indices = grasp_df[grasp_df['offset_marker'] == 1].index
            offset_times = grasp_df.loc[offset_indices, 'millis']
            plt.scatter(offset_times, [0] * len(offset_times), 
                        marker='v', color='red', s=detection_params['offset_marker_size'],
                        label='Grasp Offset', zorder=5)
            
            plt.title(f"Detected Grasp Events - {subject_only}, {grasp_type} (Label: {grasp_value})")
            plt.ylabel("State")
            plt.yticks([0, 1], ['Inactive', 'Active'])
            plt.grid(True, alpha=0.3)
            plt.legend(loc='upper right')
            
            # 4. Plot the specific sensor pattern for this grasp type (using s1)
            plt.subplot(4, 1, 4)
            if 's1_norm' in grasp_df.columns:
                plt.plot(grasp_df['millis'], grasp_df['s1_norm'], 
                         label='s1 (normalized)', color='blue', linewidth=1.5)
                
                # Highlight the active regions
                active_mask = (grasp_df['grasp_active'] > 0)
                if active_mask.any():
                    plt.fill_between(grasp_df['millis'], 
                                    -1.5, 1.5, 
                                    where=active_mask,
                                    color='green', alpha=0.1, label='Active Period')
                
                # Mark onset/offset points on this graph too
                for time in onset_times:
                    plt.axvline(x=time, color='lime', linestyle='-', alpha=0.5)
                for time in offset_times:
                    plt.axvline(x=time, color='red', linestyle='-', alpha=0.5)
                
                plt.axhline(y=-1, color='gray', linestyle='--', alpha=0.3)
                plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
                plt.axhline(y=1, color='gray', linestyle='--', alpha=0.3)
                plt.ylim(-1.5, 1.5)
            else:
                plt.text(0.5, 0.5, 's1_norm not available', 
                         horizontalalignment='center', verticalalignment='center')
            
            plt.title(f"s1 Pattern During {grasp_type} Grasp - {subject_only}")
            plt.xlabel('Time (ms)')
            plt.ylabel('s1 Value')
            plt.grid(True, alpha=0.3)
            plt.legend(loc='upper right')
            
            # Adjust layout and save
            plt.suptitle(f'Enhanced Grasp Detection: {subject_only} - {grasp_type} (Label: {grasp_value})', fontsize=16)
            plt.tight_layout()
            plt.subplots_adjust(top=0.93)
            
            # Save the figure
            filename = f"{subject_only}_{grasp_type.replace(' ', '_')}_enhanced.png"
            filepath = os.path.join(output_dir, filename)
            plt.savefig(filepath, dpi=300)
            plt.close()
            print(f"    Saved enhanced plot to {filepath}")
    
    # Combine all processed dataframes and update
    if processed_dfs:
        new_df = pd.concat(processed_dfs, axis=0)
        # Make sure the order is preserved
        new_df = new_df.sort_index()
        # Update with enhanced detection results
        subject_dataframes[subject_name] = new_df
        print(f"  Updated dataframe for {subject_only} with enhanced grasp detection")

print("\nEnhanced grasp detection complete with clear onset/offset markers.")

# Display a summary of the mapping used
print("\n----- Grasp Label Mapping Summary -----")
for grasp, label in grasp_mapping.items():
    print(f"{grasp}: {label}")

In [ ]:
# Grasp Segment Inspection and Manual Labeling Tool
print("Creating grasp segment inspection and manual labeling tool...")

# Store detected segments for each subject and grasp type
grasp_segments = {}

# Process each subject
for subject_name, df in subject_dataframes.items():
    subject_only = subject_name.split('_')[0]
    
    # Skip subjects without grasp detection results
    if 'grasp_active' not in df.columns or 'grasp_label' not in df.columns:
        print(f"⚠️ Subject {subject_only} doesn't have grasp detection results, skipping")
        continue
    
    grasp_segments[subject_only] = {}
    
    # Find all unique grasps
    unique_grasps = df['Grasp'].unique()
    
    # Process each grasp type
    for grasp_type in unique_grasps:
        # Get the data for this grasp
        grasp_df = df[df['Grasp'] == grasp_type]
        
        # Skip if no data
        if len(grasp_df) == 0:
            continue
            
        # Find onset and offset markers
        onset_indices = grasp_df[grasp_df['onset_marker'] == 1].index.tolist()
        offset_indices = grasp_df[grasp_df['offset_marker'] == 1].index.tolist()
        
        # Store segment information
        segments = []
        for i in range(len(onset_indices)):
            onset_idx = onset_indices[i]
            # Match with the corresponding offset
            offset_idx = None
            for off_idx in offset_indices:
                if off_idx > onset_idx:
                    offset_idx = off_idx
                    break
            
            # If we found a valid segment, store it
            if offset_idx is not None:
                onset_time = grasp_df.loc[onset_idx, 'millis']
                offset_time = grasp_df.loc[offset_idx, 'millis']
                duration = offset_time - onset_time
                
                segments.append({
                    'onset_index': onset_idx,
                    'offset_index': offset_idx,
                    'onset_time': onset_time,
                    'offset_time': offset_time,
                    'duration_ms': duration,
                })
        
        # Store segments for this grasp type
        if segments:
            grasp_segments[subject_only][grasp_type] = segments

# Create segment summary
print("\n===== Grasp Segment Summary =====\n")
for subject, grasps in grasp_segments.items():
    print(f"\n👤 Subject: {subject}")
    
    for grasp_type, segments in grasps.items():
        grasp_value = grasp_mapping.get(grasp_type, '?')
        print(f"\n  🤚 Grasp: {grasp_type} (Label: {grasp_value})")
        
        if not segments:
            print("     No segments detected")
            continue
            
        print(f"     Found {len(segments)} segments:")
        for i, segment in enumerate(segments):
            print(f"     #{i+1}: Index {segment['onset_index']}→{segment['offset_index']}, "
                  f"Time {segment['onset_time']:.0f}ms→{segment['offset_time']:.0f}ms, "
                  f"Duration: {segment['duration_ms']:.0f}ms")

# Create a function to visualize specific segments for a given subject and grasp type
def view_grasp_segments(subject, grasp_type, segment_indices=None):
    """
    Visualize detected segments for a specific subject and grasp type
    
    Parameters:
    - subject: Subject name (string)
    - grasp_type: Grasp type (string)
    - segment_indices: List of segment indices to highlight, or None for all
    """
    # Find the matching subject folder
    matching_folders = [name for name in subject_dataframes.keys() if name.split('_')[0] == subject]
    if not matching_folders:
        print(f"❌ Subject '{subject}' not found")
        return
    
    subject_name = matching_folders[0]
    df = subject_dataframes[subject_name]
    
    # Filter for the target grasp
    grasp_df = df[df['Grasp'] == grasp_type]
    if len(grasp_df) == 0:
        print(f"❌ Grasp '{grasp_type}' not found for subject '{subject}'")
        return
    
    # Get segments
    if subject not in grasp_segments or grasp_type not in grasp_segments[subject]:
        print(f"❌ No segments found for subject '{subject}', grasp '{grasp_type}'")
        return
    
    segments = grasp_segments[subject][grasp_type]
    
    # Filter segments if requested
    if segment_indices is not None:
        segments = [segments[i] for i in segment_indices if i < len(segments)]
    
    # Create visualization
    plt.figure(figsize=(14, 8))
    
    # Plot the activation score
    plt.subplot(2, 1, 1)
    plt.plot(grasp_df['millis'], grasp_df['smooth_activation'], 
             label='Activation Score', color='blue', alpha=0.7)
    plt.plot(grasp_df['millis'], grasp_df['detection_threshold'], 
             label='Detection Threshold', color='red', linestyle='--')
    
    # Highlight detected segments
    for i, segment in enumerate(segments):
        onset_time = grasp_df.loc[segment['onset_index'], 'millis']
        offset_time = grasp_df.loc[segment['offset_index'], 'millis']
        
        # Draw vertical lines at onset/offset
        plt.axvline(x=onset_time, color='lime', linestyle='-', 
                   label=f"Onset #{i+1}" if i == 0 else "", alpha=0.7)
        plt.axvline(x=offset_time, color='red', linestyle='-', 
                   label=f"Offset #{i+1}" if i == 0 else "", alpha=0.7)
        
        # Highlight region
        plt.axvspan(onset_time, offset_time, alpha=0.1, color='green',
                   label=f"Active Period #{i+1}" if i == 0 else "")
    
    plt.title(f"Activation Score - {subject}, {grasp_type}")
    plt.ylabel("Score")
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Plot the normalized sensor values
    plt.subplot(2, 1, 2)
    
    # Find normalized sensor columns
    norm_sensors = [col for col in grasp_df.columns if col.endswith('_norm') and col.startswith('s')][:8]
    
    for sensor in norm_sensors:
        plt.plot(grasp_df['millis'], grasp_df[sensor], 
                label=sensor.split('_')[0], alpha=0.6, linewidth=0.8)
    
    # Highlight detected segments
    for i, segment in enumerate(segments):
        onset_time = grasp_df.loc[segment['onset_index'], 'millis']
        offset_time = grasp_df.loc[segment['offset_index'], 'millis']
        
        # Draw vertical lines at onset/offset  
        plt.axvline(x=onset_time, color='lime', linestyle='-', alpha=0.7)
        plt.axvline(x=offset_time, color='red', linestyle='-', alpha=0.7)
        
        # Add text labels
        plt.text(onset_time, plt.ylim()[1]*0.9, f" #{i+1}", color='lime', fontweight='bold')
        plt.text(offset_time, plt.ylim()[1]*0.9, f" #{i+1}", color='red', fontweight='bold')
    
    plt.title(f"Normalized Sensor Values - {subject}, {grasp_type}")
    plt.xlabel("Time (ms)")
    plt.ylabel("Normalized Value")
    plt.grid(True, alpha=0.3)
    plt.legend(ncol=min(4, len(norm_sensors)))
    
    plt.tight_layout()
    plt.show()
    
    # Return the segment data for reference
    return segments

# Define a function to export segment indices for manual adjustment
def export_manual_segment_indices(subject, grasp_type):
    """
    Export segment indices in a format ready for manual adjustment
    
    Parameters:
    - subject: Subject name (string)
    - grasp_type: Grasp type (string)
    
    Returns:
    - Python code to define manual segments
    """
    if subject not in grasp_segments or grasp_type not in grasp_segments[subject]:
        print(f"❌ No segments found for subject '{subject}', grasp '{grasp_type}'")
        return None
    
    segments = grasp_segments[subject][grasp_type]
    
    # Generate Python code for manual segments
    code = f"# Manual segment indices for subject '{subject}', grasp '{grasp_type}'\n"
    code += f"manual_segments = [\n"
    
    for segment in segments:
        code += f"    ({segment['onset_index']}, {segment['offset_index']}),  # Duration: {segment['duration_ms']:.0f}ms\n"
    
    code += "]\n"
    
    print(code)
    return code

def save_all_subjects_to_csv(subject_dataframes, output_file='armband_data_export.csv'):
    """
    Save all subjects from subject_dataframes into a single CSV file.
    
    Parameters:
    -----------
    subject_dataframes : dict
        Dictionary where keys are subject folder names and values are pandas DataFrames
    output_file : str, default='armband_data_export.csv'
        Filename for the output CSV file
    
    Returns:
    --------
    str
        Path to the saved CSV file
    """
    import os
    import pandas as pd
    from datetime import datetime
    
    # Create a list to store all DataFrames
    all_dfs = []
    
    # Add a timestamp for this export
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Process each subject's DataFrame
    for subject_name, df in subject_dataframes.items():
        # Extract the short subject name
        subject_only = subject_name.split('_')[0]
        
        # Create a copy of the DataFrame to avoid modifying the original
        subject_df = df.copy()
        
        # Add subject identifier as a column
        subject_df['SubjectFolder'] = subject_name  # Full folder name
        subject_df['Subject'] = subject_only        # Short name only
        
        # Add to our collection
        all_dfs.append(subject_df)
        
    if not all_dfs:
        print("No data to save!")
        return None
    
    # Combine all DataFrames
    combined_df = pd.concat(all_dfs, ignore_index=True)
    
    # Add export timestamp
    combined_df['ExportTimestamp'] = timestamp
    
    # Check if output directory exists, if not create it
    output_dir = os.path.dirname(output_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Save to CSV
    combined_df.to_csv(output_file, index=False)
    
    # Print summary
    n_subjects = len(subject_dataframes)
    n_rows = len(combined_df)
    n_cols = len(combined_df.columns)
    
    print(f"✅ Successfully saved {n_rows} rows of data from {n_subjects} subjects ({n_cols} columns)")
    print(f"📄 File saved to: {os.path.abspath(output_file)}")
    
    return os.path.abspath(output_file)

# Example usage:
# save_all_subjects_to_csv(subject_dataframes, 'data/processed/armband_data_export.csv')

# Example usage:
print("\n===== Example Usage =====")
print("To view segments for a specific subject and grasp:")
print("    segments = view_grasp_segments('LauraPosada', 'Cilindric')")
print("\nTo view specific segments only:")
print("    segments = view_grasp_segments('LauraPosada', 'Cilindric', segment_indices=[0, 2])")
print("\nTo export segment indices for manual adjustment:")
print("    export_manual_segment_indices('LauraPosada', 'Cilindric')")

In [ ]:
subject = 'Dani'

for grasp in df.Grasp.unique():
    segments = view_grasp_segments(subject, grasp)

In [ ]:
# Function to manually update grasp segments with custom boundaries
def update_segments(subject, grasp_type, manual_segments):
    """
    Update grasp segments with manually defined boundaries
    
    Parameters:
    - subject: Subject name (string)
    - grasp_type: Grasp type (string)
    - manual_segments: List of tuples [(start_index, end_index), ...] defining the segments
    
    Returns:
    - Dictionary with updated segment information
    """
    # Find the matching subject folder
    matching_folders = [name for name in subject_dataframes.keys() if name.split('_')[0] == subject]
    if not matching_folders:
        print(f"❌ Subject '{subject}' not found")
        return None
    
    subject_name = matching_folders[0]
    df = subject_dataframes[subject_name]
    
    # Filter for the target grasp
    grasp_df = df[df['Grasp'] == grasp_type]
    if len(grasp_df) == 0:
        print(f"❌ Grasp '{grasp_type}' not found for subject '{subject}'")
        return None
    
    # Create new segments from the manual definitions
    updated_segments = []
    for i, (start_idx, end_idx) in enumerate(manual_segments):
        # Validate indices
        if start_idx not in grasp_df.index or end_idx not in grasp_df.index:
            print(f"⚠️ Segment #{i+1} has invalid indices: ({start_idx}, {end_idx})")
            continue
            
        # Ensure start comes before end
        if start_idx >= end_idx:
            print(f"⚠️ Segment #{i+1} has start index >= end index: ({start_idx}, {end_idx})")
            continue
        
        # Get timing information
        onset_time = grasp_df.loc[start_idx, 'millis']
        offset_time = grasp_df.loc[end_idx, 'millis']
        duration = offset_time - onset_time
        
        # Create segment dictionary
        segment = {
            'onset_index': start_idx,
            'offset_index': end_idx,
            'onset_time': onset_time,
            'offset_time': offset_time,
            'duration_ms': duration,
        }
        updated_segments.append(segment)
    
    # Store the updated segments
    if subject not in grasp_segments:
        grasp_segments[subject] = {}
    
    grasp_segments[subject][grasp_type] = updated_segments
    
    # Print summary
    print(f"✅ Updated segments for {subject}, {grasp_type}:")
    for i, segment in enumerate(updated_segments):
        print(f"  #{i+1}: Index {segment['onset_index']}→{segment['offset_index']}, "
              f"Time {segment['onset_time']:.0f}ms→{segment['offset_time']:.0f}ms, "
              f"Duration: {segment['duration_ms']:.0f}ms")
    
    # Automatically visualize the updated segments
    print("\nShowing updated segments visualization:")
    view_grasp_segments(subject, grasp_type)
    
    return updated_segments

# Example usage:
# Define custom segments for a subject and grasp type
# update_segments('Dani', 'Cilindric', [
#     (12345, 12500),  # First segment: (start_index, end_index)
#     (13000, 13150),  # Second segment
# ])

### Manual relabaling process starts!

Grasp by grasp start and end indexes are adjusted until they match visual confirmation.

Every so often saving the latest dataframe is recommended.

In [ ]:
subject = 'Dani'
grasp = 'Handle'
# 'CloseHand' or 'Handle' or 'Pinch' or 'PointTripod' or 'Tripod' or 'Cilindric'

view_grasp_segments(subject, grasp)
export_manual_segment_indices(subject, grasp)

In [ ]:
manual_segments = [
    (5520, 5725),  # Duration: 11300ms
    (5945, 6145),  # Duration: 12450ms
    (6355, 6601),  # Duration: 12300ms
    (6771, 7008),  # Duration: 11850ms
    (7200, 7428),  # Duration: 11400ms
]

# Update with your custom segments
new_segments = update_segments(subject, grasp, manual_segments)


In [ ]:
# Save all processed data to a CSV file
output_path = save_all_subjects_to_csv(subject_dataframes, 'data/armband_data_export.csv')

In [ ]:
subject_dataframes["Dani_16 08 2024"]